In [1]:
# Mounting Drive for getting dataset
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
import os.path
import os
import glob as gb

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from keras.models import Model
from keras.layers import Input, Dense, Concatenate

In [18]:
# Load the first model
model1 = tf.keras.models.load_model('/content/drive/MyDrive/Capstone_Project/Models/MobileNet_v2.h5')

# Load the second model
model2 = tf.keras.models.load_model('/content/drive/MyDrive/Capstone_Project/Models/ResNet.h5')

In [4]:
train_df = '/content/drive/MyDrive/Capstone_Project/Data/Train'
test_df = '/content/drive/MyDrive/Capstone_Project/Data/Test'

In [21]:
train_datagen = ImageDataGenerator(rescale = 1./255, 
                                   shear_range = 0.2, #slants the shape of the object - normal shearing
                                   zoom_range = 0.2, #zooms the image
                                   horizontal_flip = True) #flips both rows and coloumns horizontally

In [22]:
# Create a new input layer for the ensemble model
input_layer = Input(shape=(224,224,3))

# Connect the input layer to both models
output1 = model1(input_layer)
output2 = model2(input_layer)

In [23]:
# Concatenate the output of the two models
output = Concatenate()([output1, output2])

# Add a final dense layer to make the predictions
output = Dense(9, activation='softmax', name="dense_a")(output)

# Create the ensemble model
ensemble_model = Model(inputs=input_layer, outputs=output)

# Compile the ensemble model
ensemble_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [24]:
train_generator = train_datagen.flow_from_directory(
    train_df,
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=16
)

Found 63 images belonging to 9 classes.


In [25]:
history = ensemble_model.fit(
    train_generator,
    epochs=20
)

Epoch 1/20
4/4 [==============================] - 59s 11s/step - loss: 2.2058 - accuracy: 0.1111
Epoch 2/20
4/4 [==============================] - 47s 12s/step - loss: 2.1399 - accuracy: 0.1746
Epoch 3/20
4/4 [==============================] - 50s 13s/step - loss: 2.1250 - accuracy: 0.2698
Epoch 4/20
4/4 [==============================] - 45s 11s/step - loss: 2.1361 - accuracy: 0.1587
Epoch 5/20
4/4 [==============================] - 45s 11s/step - loss: 2.1543 - accuracy: 0.1746
Epoch 6/20
4/4 [==============================] - 45s 11s/step - loss: 2.1056 - accuracy: 0.1905
Epoch 7/20
4/4 [==============================] - 49s 12s/step - loss: 2.1327 - accuracy: 0.1746
Epoch 8/20
4/4 [==============================] - 45s 11s/step - loss: 2.0563 - accuracy: 0.2381
Epoch 9/20
4/4 [==============================] - 45s 11s/step - loss: 2.0759 - accuracy: 0.2381
Epoch 10/20
4/4 [==============================] - 45s 11s/step - loss: 2.0746 - accuracy: 0.2222
Epoch 11/20
4/4 [============

In [26]:
test_datagen = ImageDataGenerator(rescale = 1./255)

test_generator = test_datagen.flow_from_directory(
    test_df,
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=16
)

Found 27 images belonging to 9 classes.


In [27]:
results = ensemble_model.evaluate(test_generator)
print("Test Accuracy: {:.2f}%".format(results[1] * 100))

2/2 [==============================] - 7s 2s/step - loss: 2.1314 - accuracy: 0.1852
Test Accuracy: 18.52%


In [28]:
ensemble_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_8 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 model (Functional)             (None, 9)            2439625     ['input_8[0][0]']                
                                                                                                  
 resnet50 (Functional)          (None, 9)            23606153    ['input_8[0][0]']                
                                                                                                  
 concatenate_2 (Concatenate)    (None, 18)           0           ['model[1][0]',            